In [8]:
import numpy as np
import os
import cv2
import random

DATADIR = "Project_Images"
CATEGORIES = ["ng", "ok"]
IMG_SIZE = 299
RESIZE_DIR = "resized_imgs"

if os.path.exists(RESIZE_DIR):
    print("[INFO] Directories already existed...")
else:
    os.mkdir(RESIZE_DIR)
    os.mkdir(RESIZE_DIR+'/ok')
    os.mkdir(RESIZE_DIR+'/ng')
    print("[INFO] Directories have been created...")

[INFO] Directories already existed...


In [ ]:
training_data = []
def roi(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
                
    # Find contour and sort by contour area
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # Find bounding box and extract ROI
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ROI = image[y:y+h, x:x+w]
        break                
    new_array = cv2.resize(ROI, (IMG_SIZE, IMG_SIZE),interpolation = cv2.INTER_AREA)
    return new_array

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = roi(img_array)

                new_path = RESIZE_DIR+"/"+category
                new_img = "ROI_"+img
                resize_path = os.path.join(new_path, new_img)
                cv2.imwrite(resize_path, new_array)
                training_data.append([new_array, class_num])
                print("[INFO] New image has been saved: " + new_img)
            except Exception as e:
                print("[WARN] "+ str(e))
                pass
create_training_data()

- Se genera ROI
- guarda en directorio para resized img en las categorias correspondientes
- logging info

In [3]:
random.shuffle(training_data)

In [4]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

In [5]:
import pickle
pickle_out = open("X_2.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y_2.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()